In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square
from time import sleep

In [3]:
from netgen.occ import *
rect = MoveTo(-1,-1).Rectangle (2,2).Face()
rect.edges.Min(X).name="left"
shape = rect
mesh = Mesh (OCCGeometry( shape, dim=2).GenerateMesh(maxh=0.03)).Curve(5)
Draw (shape)
Draw (mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [4]:
tau = 0.005
tend = 2
u0 = exp(-400*(x**2 + y**2)) 
v0 = 0

fes = H1(mesh, order=3)
u,v = fes.TnT()
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()
mstar = BilinearForm(mform+tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")
f = LinearForm(fes).Assemble()

In [5]:
gfu = GridFunction(fes)
gfv = GridFunction(fes)

gfu.Set(u0)
gfv.Set(v0)

scene = Draw (gfu, deformation=True, order=3)

for j in range(int(tend/tau)):
    gfu.vec.data += tau/2 * gfv.vec
    gfv.vec.data += tau * mstarinv * (f.vec - a.mat * gfu.vec)
    gfu.vec.data += tau/2 * gfv.vec

    if j%2 == 0:
        scene.Redraw()
        # sleep(0.1)
scene.Redraw()

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [6]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import unit_square # Not strictly used, but kept from original
from time import sleep
from netgen.occ import *

# 1. Define the geometry (no changes needed here for basic ABC)
# The boundary rect.edges.Min(X).name="left" is not strictly necessary
# for applying ABC on all boundaries if using ds() broadly, but doesn't hurt.
# For clarity, if you wanted to ensure names for all boundaries to use explicitly:
# rect = MoveTo(-1,-1).Rectangle (2,2).Face()
# rect.edges.Min(X).name="left"
# rect.edges.Max(X).name="right"
# rect.edges.Min(Y).name="bottom"
# rect.edges.Max(Y).name="top"
# However, ds() without definedon will apply to all boundaries of the domain.
rect = MoveTo(-1,-1).Rectangle (2,2).Face()
rect.edges.Min(X).name="left" # This naming is kept but not essential for below ABC
shape = rect
mesh = Mesh (OCCGeometry( shape, dim=2).GenerateMesh(maxh=0.03)).Curve(5)

Draw (shape) # Draw the geometry shape
Draw (mesh); # Draw the mesh

tau = 0.005
tend = 2

# Initial conditions
u0 = exp(-400*(x**2 + y**2))
v0 = 0

# Finite element space
fes = H1(mesh, order=3)
u,v = fes.TnT() # u is trial, v is test (or vice-versa, consistently)

# Bilinear forms for mass and stiffness matrices
mform = u*v*dx
aform = grad(u)*grad(v)*dx

m = BilinearForm(mform).Assemble()
a = BilinearForm(aform).Assemble()

# 2. Define the bilinear form for the absorbing boundary condition
# This term corresponds to integral of v*w*ds on the boundary
# Here, 'u' acts as the test function 'w', and 'v' as the trial function for 'v' itself.
# ds() without a 'definedon' argument integrates over all boundaries of the domain.
# If you have specifically named all your boundaries (e.g. "left", "right", "top", "bottom"),
# you could use: ds(definedon=mesh.Boundaries("left|right|top|bottom"))
# Or more generally for all boundaries: ds(definedon=mesh.Boundaries(".*"))
c_wave_speed = 1.0 # Wave speed, assumed to be 1
bform_abc = (1/c_wave_speed) * u*v*ds # u is test, v is trial from fes.TnT()

b_abc = BilinearForm(fes) # Important: use fes, not just bform_abc directly
b_abc += bform_abc      # Add the symbolic form to the BilinearForm object
b_abc.Assemble()

# Modified mass matrix for the time integrator
mstar = BilinearForm(mform+tau**2/4*aform).Assemble()
mstarinv = mstar.mat.Inverse(inverse="sparsecholesky")

# Linear form (e.g., for sources, not used here as f.vec is zero)
f = LinearForm(fes).Assemble() # f.vec will be a zero vector

# Grid functions for solution u and its time derivative v
gfu = GridFunction(fes)
gfv = GridFunction(fes)

gfu.Set(u0)
gfv.Set(v0)

scene = Draw (gfu, deformation=True, order=3) # deformation uses gfu for displacement

# Time loop
for j in range(int(tend/tau)):
    # First half-step for u
    gfu.vec.data += tau/2 * gfv.vec

    # Update for v (with absorbing boundary condition)
    # Original: res_v = f.vec - a.mat * gfu.vec
    # Modified: include the damping term -b_abc.mat * gfv.vec
    # This implements M* dv/dt = -A*u - B_abc*v
    residual_v = f.vec.CreateVector() # ensure f.vec is not modified if it's non-zero later
    residual_v.data = f.vec - a.mat * gfu.vec - b_abc.mat * gfv.vec
    gfv.vec.data += tau * mstarinv * residual_v

    # Second half-step for u
    gfu.vec.data += tau/2 * gfv.vec

    if j%20 == 0: # Redraw less frequently for speed
        scene.Redraw()
        # sleep(0.01) # Optional: slow down for visualization

scene.Redraw() # Final redraw

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

used dof inconsistency
(silence this warning by setting BilinearForm(...check_unused=False) )


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

/home/hpicatto/noASCII/tuwien/ASC-bla/.pixi/envs/research-intro-sc/lib/python3.13/site-packages/netgen/webgui.py:20: RuntimeWarning: overflow encountered in cast
  values = np.array(data.flatten(), dtype=dtype)
